Instanzieren der Klasse DatabaseMeta in der alle Metadaten der Datenbank gespeichert sind.

In [1]:
from sqlalchemy import text
from DatabaseMeta import DatabaseMeta
dbm = DatabaseMeta("sqlite:///test.db", verbose_mode=False)



Prüfung erfolgreich abgeschlossen!


Instanzieren der Klasse ETLPipeline; dem Konstruktor wird das DatabaseMeta-Objekt übergeben.

In [2]:

from ETL import ETLPipeline
etl = ETLPipeline(dbm)

Um die Dimensionstabellen mit den Ausgangsdaten zu befüllen,
werden zunächst mehrere csv-Dateien in ein Pandas-Dataframe eingelesen.
Die Dateien liegen im Ordner 'dim_data' und
beinhalten die Ausgangsdaten für die Dimensionstabellen 'DT_Produkt', 'DT_Personal' und 'DT_Kunde'.

In [3]:
import pandas as pd
produkte_col_types = {   "kategorie": str,
                "produkt_code": str,
                "name": str,
                "variante": str,
                "hersteller_artikel_code": str,
                "hersteller_code": str,
                "hersteller_name": str,
                "Verkaufspreis_Brutto": str,
                "Verkaufspreis_MwSt": str,
                "hersteller_plz": str,
                "hersteller_ort": str,
                "hersteller_land": str}
csv_file = './dim_data/produkte.csv'
produkte = pd.read_csv(csv_file, dtype=produkte_col_types)
produkte["Verkaufspreis_Brutto"] = pd.to_numeric(produkte["Verkaufspreis_Brutto"])
produkte["Verkaufspreis_MwSt"] = pd.to_numeric(produkte["Verkaufspreis_MwSt"])
produkte["verkaufspreis_netto"] = produkte["Verkaufspreis_Brutto"]-produkte["Verkaufspreis_MwSt"]
produkte["verkaufspreis_netto"] = pd.to_numeric(produkte["verkaufspreis_netto"])
produkte["verkaufspreis_netto"] = produkte["verkaufspreis_netto"].round(2)
produkte = produkte.drop(labels=["Verkaufspreis_Brutto","Verkaufspreis_MwSt"], axis=1)
produkte = produkte.drop(labels=["datum","uhrzeit"], axis=1)

kunden_col_types = {   "kunde_code": str,
                "name": str,
                "vorname": str,
                "firma": str,
                "plz": str,
                "ort": str,
                "land": str}
csv_file = './dim_data/kunden.csv'
kunden = pd.read_csv(csv_file, dtype=kunden_col_types)

personal_col_types = {   "personal_code": str,
                "name": str,
                "vorname": str,
                "abteilung": str,
                "abteilung_vorher": str}
csv_file = './dim_data/personal.csv'
personal = pd.read_csv(csv_file, dtype=personal_col_types)
personal = personal.drop(labels=["datum"], axis=1)

Im nächsten Schritt werden die DataFrames mit den Ausgangsdaten mittels der Funktion 'change_dim_value' einzeln
der ETLPipeline übergeben. Die Pipeline speichert die Daten in der Datenbank.
Der Gültigkeitsbeginn der Ausgangsdaten wird auf das frühestmögliche Datum gesetzt 'date.min' (dritter Parameter).

In die Dimensionstabelle 'DT_Personal' sollen Werte in die Spalte 'abteilung_vorher' gespeichert werden.
Diese Spalte ist für die Aufnahme von SCD-Typ 3 Werten markiert.
Daher muss der optionale Parameter 'scd1' True gesetzt werden. Damit können in allen Spalten Werte überschrieben werden.

Um die Änderungen zu Verfolgen sollte ein Tool der Wahl genutzt werden, um die (SQLite-)Datenbank auszulesen.

In [35]:
from datetime import date
for i in range(produkte.shape[0]):
    etl.change_dim_value('DT_Produkt', produkte.iloc[i].to_dict(), date.min)

for i in range(kunden.shape[0]):
    etl.change_dim_value('DT_Kunde', kunden.iloc[i].to_dict(), date.min)

for i in range(personal.shape[0]):
    etl.change_dim_value('DT_Personal', personal.iloc[i].to_dict(), date.min, scd1=True)

/home/daniel/anaconda3/lib/python3.8/site-packages/sqlalchemy/sql/sqltypes.py:724: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werden nun einfach überschrieben!
ACHTUNG: Werte werde

Instanziere die Klasse TestDataGenerator. Dem Konstruktor ist ein dict zu übergeben, dass den Dimensionen eine
csv-Datei zuweist, in der sich die Dimensionsdaten befinden.

In [4]:
from TestDataGenerator import TestDataGenerator
tdg = TestDataGenerator({'personal':'./test_data/personal_original.csv',
                         'kunde':'./test_data/kunden_original.csv',
                         'produkt':'./test_data/produkte_original.csv'})

Nun können mit dem TestDataGenerator Faktendaten erzeugt werden.
Die Methode 'dump_data' erzeugt eine csv-Datei mit den Testdaten. Als Parameter ist die Anzahl an Datenreihen anzugeben.
Die Methode gibt den erzeugten Dateinamen zurück, die Datei wird im Ordner 'testData' abgespeichert.

In [37]:
# Code-Zelle TD1
file_path = tdg.dump_data(1000)

Die csv-Datei wird der ETLPipeline übergeben. Die Fakten werden in der Datenbank gespeichert.

In [38]:
# Code-Zelle TD2
etl.etl_fact_csv(file_path)

True

Die Code-Zellen TD1 und TD2 können nun beliebig oft wiederholt werden, um weitere Testdaten in die Datenbank zu laden.

Im Folgenden werden csv-Dateien eingelesen, die Veränderungen (mit Zeitstempel) von Dimensionswerten enthalten.

In die Tabelle 'DT_Kunde' werden neue Datensätze eingefügt:

In [40]:
etl.etl_dim_csv('./test_data/kunden_neu.csv')

Um die neuen Kunden bei der Generierung von weiteren Faktendaten zu berücksichtigen,
wird die neue Datenbasis dem 'TestDataGenerator'-Objekt "tdg" übergeben.
Dazu muss diese zuvor aus der Tabelle 'DT_Kunde' exportiert werden.
Dafür wird die Helferfunktion "dump_dim_data" definiert.

In [5]:
def dump_dim_data(table:str, file_name:str) -> str:
    with etl._ETLPipeline__engine.connect() as conn:
        dim_DF = pd.read_sql_table(table,conn)
        dim_DF = dim_DF.drop('id',axis=1)  # die technische ID wird nicht benötigt
        full_file_path = 'dim_data_drops/' + file_name + '.csv'
        dim_DF.to_csv(full_file_path,index=False)
        return full_file_path

Die Funktion gibt den Dateipfad zur gespeicherten csv-Datei zurück.
Dieser wird dann dem 'TestDataGenerator'-Objekt "tdg" übergeben.

In [6]:
dump_file = dump_dim_data('DT_Kunde', 'DT_Kunde__neue_Kunden')
tdg.change_data({'kunde': dump_file})
file_path = tdg.dump_data(1000)
etl.etl_fact_csv(file_path)

/home/daniel/anaconda3/lib/python3.8/site-packages/sqlalchemy/sql/sqltypes.py:724: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


True

SCD Typ 1:

Bei einigen Kunden haben sich der Wohnort oder Nachname geändert.
Die neuen Werte überschreiben die alten Werte in der Tabelle.

In [9]:
etl.etl_dim_csv('./test_data/kunden_changes.csv')

Diese Änderungen müssen nicht dem 'TestDataGenerator'-Objekt "tdg" übergeben werden.


SCD Typ 2:

Am 01.07.2020 trat die temporäre Absenkung der Mehrwertsteuer von 19 % auf 16 % in Kraft.
Daher ändert sich ab diesem Tag der Verkaufspreis der Produkte.
Außerdem ändern sich einige Variantenbezeichnungen und es kommen neue Produkte hinzu.

In [10]:
etl.etl_dim_csv('./test_data/produkte_changes.csv')

Diese Daten werden wiederum dem 'TestDataGenerator'-Objekt "tdg" übergeben.

In [6]:
dump_file = dump_dim_data('DT_Produkt', 'DT_Produkt__aenderungen')
tdg.change_data({'produkt': dump_file})

Neu erzeugte Faktendaten können jedoch erst nach Beginn der Gültigkeit die neuen Datensätze referenzieren!
Daher wird das Datum in den November 2020 geändert.

In [7]:
from datetime import datetime
tdg.skip_to_date(datetime(2020,11,1))
file_path = tdg.dump_data(1000)
etl.etl_fact_csv(file_path)

Datum geändert zu 2020-11-01T00:00:00


/home/daniel/anaconda3/lib/python3.8/site-packages/sqlalchemy/sql/sqltypes.py:724: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


True

Besonderheit Star-Schema:

In der Tabelle 'DT_Produkt' sind auch Herstellerdaten gespeichert.
Die Tabelle ist lediglich in der ersten Normalform.
Daher müssen Änderungen, die Herstellerdaten betreffen, in allen entsprechenden Datensätzen geändert werden.

Beim Snowflake-Schema liegen die Tabellen in normalisierter Form vor. Daher tritt diese Besonderheit dort nicht auf.

In [8]:
etl.change_dim_value('DT_Produkt',{'produkt_code': '445862-OW', 'hersteller_name': 'Squeeler'})

45

Bei der Änderung des Herstellernamens in einem Datensatz werden alle Datensätze geändert, die den gleichen
'hersteller_code' enthalten.
Diese Änderung führt hier nicht zu einer Historisierung (dies ließe sich jedoch so programmieren).

SCD Typ 3:

Einige Mitarbeiter haben die Abteilung gewechselt.
Die bisherige Abteilung wird in der Spalte 'abteilung_vorher' gespeichert.
Die neue Abteilung wird in 'abteilung' gespeichert.
Alte Werte werden von neuen Werten "herausgeschoben".

In [9]:
etl.etl_dim_csv('./test_data/personal_changes.csv')

abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher
abteilung in DT_Personal zu abteilung_vorher


Zusatz SCD Typ 3:

Durch eine Zusammenführung von bisher getrennten
Personalwirtschaftssystemen sind neue Personal-Code erforderlich geworden.
Da der Zeitpunkt dieser Maßnahme bekannt ist und ein solcher Wechsel nicht wieder erwartet wird,
jedoch die alten Personal-Code weiterhin gespeichert bleiben sollen, wird dies mit dem SCD Typ 3 abgebildet.

Da die alten Personal-Code eineindeutig sind, identifizieren diese auch noch jeden Datensatz.
Die erforderliche Mapping-Tabelle von alten zu neuen Personal-Code besteht also nur aus jeweils diesen beiden.

In der Tabelle 'DT_Personal' wird die Spalte 'personal_code_neu' ergänzt.
Da in diesem Prototyp noch keine Versionierung der Datenbank vorgenommen wird, werden nun reine SQL-Befehle genutzt.

In [10]:
with etl._ETLPipeline__engine.connect() as conn:
    conn.execute('ALTER TABLE DT_Personal ADD personal_code_neu TEXT')

Sollen die neuen Personal-Codes über die ETL-Pipeline eingepflegt werden, müsste zunächst das DatabaseMeta-Objekt angepasst werden.
Hier nun der direkte Weg mit Hilfe des UPDATE-Befehl (sql).

In [11]:
from sqlalchemy import text

with open('./test_data/personal_mapping.csv','r') as map_file:
    map_df = pd.read_csv(map_file,dtype=str)
    with etl._ETLPipeline__engine.connect() as conn:
        for i in range(map_df.shape[0]):
            stmt = text('UPDATE DT_Personal SET personal_code_neu = :x WHERE personal_code = :y')
            stmt = stmt.bindparams(x = map_df.iloc[i]["personal_code_neu"], y = map_df.iloc[i]["personal_code"])
            conn.execute(stmt)

Nun wird die Spalte 'personal_code' in 'personal_code_alt' umbenannt
und die Spalte 'personal_code_neu' in 'personal_code'.
Das oben beschriebene Verhalten der Abteilungsspalten (herausschieben von neuen Daten) wird hier nicht implementiert,
da die Personal-Code sich grundsätzlich nicht ändern.

In [12]:
with etl._ETLPipeline__engine.connect() as conn:
    conn.execute('ALTER TABLE DT_Personal RENAME COLUMN personal_code TO personal_code_alt')
    conn.execute('ALTER TABLE DT_Personal RENAME COLUMN personal_code_neu TO personal_code')

Diese Änderung im Tabellenschema wird nun noch der ETL-Pipeline mitgegeben.
Dazu wird das DatabaseMeta-Objekt geändert.

In [13]:
from sqlalchemy import Table, Column, Integer, String
etl.dbm.DT_employee_table = Table("DT_Personal", etl.dbm.metadata,
                                        Column('personal_code_alt', String),
                                        extend_existing=True
                                       )

Zuletzt wird die Datenbasis des TestDataGenerators geändert.

In [14]:
dump_file = dump_dim_data('DT_Personal', 'DT_Personal__neue_personalcode')
tdg.change_data({'personal': dump_file})
file_path = tdg.dump_data(1000)
etl.etl_fact_csv(file_path)

True